In [1]:
#!python3
import csv
import sys

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

plt.rcParams['ps.useafm'] = True
plt.rcParams['pdf.use14corefonts'] = True
plt.rcParams['text.usetex'] = True

def float_or_na(s):
    if (s == "NA"):
        return 0
    else:
        return float(s)

# input: csv data as triples of (tag, test_name, running_time)
data = []
data2 = []
min_times = {}
with open("./synthetic.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        n = int(row[0])
        k = int(row[1])
        parlay = float_or_na(row[2])
        hash_build = float(row[3])
        hash_query = float(row[4])-hash_build
        block_build = float(row[5])
        block_query = float(row[6])-block_build
        sa_build = float(row[7])
        sa_query = float(row[8])-sa_build
        dac_mm = float_or_na(row[9])

        data.append([n, k, "BFS-B-Hash: Query", block_build + block_query])
        data.append([n, k, "BFS-Hash: Query", hash_build + hash_query])
        data.append([n, k, "BFS-SA: Query", sa_build + sa_query])
        data.append([n, k, "DaC-SP", dac_mm])
        data.append([n, k, "Parlay", parlay])

        data2.append([n, k, "BFS-B-Hash: Building", block_build])
        data2.append([n, k, "BFS-Hash: Building", hash_build])
        data2.append([n, k, "BFS-SA: Building", sa_build])
        data2.append([n, k, "DaC-SP", dac_mm])
        data2.append([n, k, "Parlay", parlay])

full_df = pd.DataFrame(data, columns = ['N', 'K', 'Algorithm', 'Time'])
full_df2 = pd.DataFrame(data2, columns = ['N', 'K', 'Algorithm', 'Time'])

full_df = full_df.sort_values(by=["N", "K"])
full_df2 = full_df2.sort_values(by=["N", "K"])

def setnumformat(x):
    print(x)
    if (x < 0.1):
        st = '{:.2f}'.format(x)
    else:
        if (x>100):
            st = '{:.3g}'.format(x)
        else:
            st = '{:.2g}'.format(x)
    print("here1 = " + st)
    if st[0] == '0':
        st = st[1:]
    print("here2 = " + st)
    return st
  

def show_values_on_bars(axs, df, ct):
    def _show_on_single_plot(ax, df, ct):
        for p in ax.patches:
            _x = (p.get_x()+p.get_width() / 2)
            total_time = df.iloc[ct]['Time']
            building_time = df2.iloc[ct]['Time']
            if (total_time > y_max):
                if df.iloc[ct]['Algorithm'] == "BFS-B-Hash: Query" or df.iloc[ct]['Algorithm'] == "BFS-Hash: Query" or df.iloc[ct]['Algorithm'] == "BFS-SA: Query":
                    building_time = df_k2.iloc[ct]['Time']
                    query_time = total_time-building_time
                    #value = '{:.2g}'.format(building_time) + "$+$" '{:.2g}'.format(query_time)
                    value = setnumformat(building_time) + "$+$" + setnumformat(query_time)
                else:
                    #value = '{:.2g}'.format(total_time)
                    value = setnumformat(total_time)

                _y = y_max * 0.15
                ct = ct + 1
                ax.text(_x, _y, value, ha="center", color='lightcyan', rotation = 90, fontsize="12")
            else:
                if (total_time == float(0)):
                    value = "x"
                    _y = 0
                    ax.text(_x, _y, value, ha="center", color='darkred', fontsize="15")
                ct = ct + 1

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax, df, ct)
    else:
        _show_on_single_plot(axs, df, ct)

sizes = full_df["N"]
print(set(sizes))

def getmathformat(x):
    st = str(x)
    if (str(x) == '1000000000'):
        st = "$10^9$"
    if (str(x) == '100000000'):
        st = "$10^8$"
    if (str(x) == '10000000'):
        st = "$10^7$"
    if (str(x) == '1000000'):
        st = "$10^6$"  
    if (str(x) == '100000'):
        st = "$10^5$"  
    if (str(x) == '10000'):
        st = "$10^4$"  
    if (str(x) == '1000'):
        st = "$10^3$"  
    if (str(x) == '100'):
        st = "$10^2$"  
    return st


for size in set(sizes):
    df = full_df.loc[full_df["N"] == size]
    df2 = full_df2.loc[full_df2["N"] == size]
    plt.figure()
    sns.set(font_scale=1)
    plt.style.use(['seaborn-dark', 'seaborn-paper'])
    #set_style()
    pal_sets = sns.color_palette("viridis_r", n_colors=10)
    # pal = sns.color_palette("blend:#7AB,#9f9b9a", n_colors=5)
    # pal2 = sns.color_palette("blend:#008080,#cd7f32", n_colors=5)
    # pal2 = ["#a1cfe6","#914e75","#b88d00","#918fc2","#749e79"] #light
    # pal = ["#1c6884","#ede0e7","#eed7b1","#edecf5","#e7eee8"] #dark
    pal = []
    pal2 = []
    for i in range(0, 5):
        pal.append(pal_sets[2* i])
        pal2.append(pal_sets[i * 2 + 1])
    


    ks = sorted(set(df["K"]))
    num_k = len(ks)
    print("size=" + str(size))
    print(ks)
    fig, axes = plt.subplots(1, 6, figsize=(11,1.5))

    for idx, k in enumerate(ks):
        df_k = df.loc[df["K"] == k]
        df_k2 = df2.loc[df2["K"] == k]
        print("df_k")
        print(df_k)
        print("df_k2")
        print(df_k2)
        times = df_k.loc[(df_k["N"] == size) & ((df_k["Algorithm"] == "BFS-Hash: Query") | (df_k["Algorithm"] == "BFS-B-Hash: Query"))]["Time"]
        all_times = df_k.loc[(df_k["N"] == size) & ((df_k["Algorithm"] == "BFS-SA: Query") | (df_k["Algorithm"] == "DaC-SP") | (df_k["Algorithm"] == "Parlay"))]["Time"]
        all_times = filter(lambda x: x > 0, all_times)
        max_time = max(times)
        other_time_min = min(all_times)
        print("other-min=" + str(other_time_min))
        y_max = max_time * 1.5
        if other_time_min < y_max * 2.5:
            y_max = max(y_max, other_time_min * 1.1)
        g = sns.barplot(ax=axes[idx], x='K', y='Time', hue='Algorithm', data=df_k, palette=pal, edgecolor=".3")
        g.set(xlabel=None, ylabel=None)
        axes[idx].set_ylim(0, y_max)
        axes[idx].legend([],[], frameon=False)

        ct = 0
        show_values_on_bars(axes[idx], df_k, ct)
        for p in axes[idx].patches:
            w = p.get_width()

        g = sns.barplot(ax=axes[idx], x='K', y='Time', hue='Algorithm', data=df_k2, palette=pal2, edgecolor="0.3")
        g.set(xlabel=None, ylabel=None)
        axes[idx].set_ylim(0, y_max)
        axes[idx].legend([],[], frameon=False)
        
        axes[idx].tick_params('y', labelsize=14)
        
        #for p in axes[idx].patches:
        #    p.set_width(w)
        
        """
        if (str(size) == '1000000000'):
            plt.sca(axes[idx])
            axes[idx].tick_params('x', labelsize=14)
            plt.xticks([0], ['k=' + str(k)])
            #axes[idx].set(xlabel='x-label', ylabel='y-label')
            #axes[idx].legend([g[0]],["a"], frameon=False)
        else:
            #axes[idx].tick_params('x', labelsize=0)
            plt.sca(axes[idx])
            plt.xticks([])
        """
        if (int(str(size)) > 3):
            plt.sca(axes[idx])
            axes[idx].tick_params('x', labelsize=14)
            plt.xticks([0], ['n=' + getmathformat(size) + '  k=' + getmathformat(k)])
        if (str(size) == '0'):
            st = ""
            kk = int(str(k))
            if kk > 4:
                st = "Wiki v"
                kk = kk-4
            else: 
                st = "Linux v"
                kk = kk-1
            plt.sca(axes[idx])
            axes[idx].tick_params('x', labelsize=14)
            print("st=" + st)
            plt.xticks([0], [st + str(kk)])
        if (str(size) == '1'):
            st = "DNA Pair "
            kk = int(str(k))
            kk = kk-1
            plt.sca(axes[idx])
            axes[idx].tick_params('x', labelsize=14)
            plt.xticks([0], [st + str(kk)])

        
    if num_k == 3:
        axes[3].axis('off')
        axes[4].axis('off')
        axes[5].axis('off')
        handles, labels = axes[0].get_legend_handles_labels()
        labels_out = labels[: 8]
    # subfigs[1,1].legend(handles, labels, loc = (0.1, 0.1), ncol=2, fontsize = 15)
        legend1 = plt.legend(handles, labels_out, bbox_to_anchor=(5.3,1.2), fontsize=14, frameon=False, ncol=2)
        # legend1 = plt.legend(bbox_to_anchor=(5.3,1.2), fontsize=14, frameon=False,ncol=2)
        plt.gca().add_artist(legend1)
        
    plt.subplots_adjust(wspace = 0.5, hspace = 0)
        #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
        #fig.supxlabel("Number of edits")
        #fig.supylabel("Time (s)")

        #plt.text(-9, 5, st, fontsize=15)

    fig.tight_layout()
    plt.savefig("output/synthetic/" + str(size) + ".pdf")

    plt.clf()
        

{0, 1, 1000000, 10000000, 100000000, 1000000000}
size=0
[2, 3, 4, 5, 6, 7]
df_k
     N  K          Algorithm       Time
120  0  2  BFS-B-Hash: Query   0.009106
121  0  2    BFS-Hash: Query   0.011752
122  0  2      BFS-SA: Query   0.111508
123  0  2             DaC-SP  28.537341
124  0  2             Parlay   7.055708
df_k2
     N  K             Algorithm       Time
120  0  2  BFS-B-Hash: Building   0.001137
121  0  2    BFS-Hash: Building   0.004369
122  0  2      BFS-SA: Building   0.103821
123  0  2                DaC-SP  28.537341
124  0  2                Parlay   7.055708
other-min=0.111508
0.103821
here1 = 0.1
here2 = .1
0.007686999999999999
here1 = 0.01
here2 = .01
28.537341
here1 = 29
here2 = 29
7.055708
here1 = 7.1
here2 = 7.1
st=Linux v
df_k
     N  K          Algorithm      Time
125  0  3  BFS-B-Hash: Query  0.175605
126  0  3    BFS-Hash: Query  0.178219
127  0  3      BFS-SA: Query  0.248973
128  0  3             DaC-SP  0.000000
129  0  3             Parlay  6.873640
df_k

<ipython-input-1-f963fe113f08>:134: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-dark', 'seaborn-paper'])
<ipython-input-1-f963fe113f08>:134: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-dark', 'seaborn-paper'])
<ipython-input-1-f963fe113f08>:134: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, 

size=1
[2, 3, 4]
df_k
     N  K          Algorithm      Time
150  1  2  BFS-B-Hash: Query  0.057211
151  1  2    BFS-Hash: Query  0.062563
152  1  2      BFS-SA: Query  2.975269
153  1  2             DaC-SP  0.000000
154  1  2             Parlay  0.000000
df_k2
     N  K             Algorithm      Time
150  1  2  BFS-B-Hash: Building  0.009682
151  1  2    BFS-Hash: Building  0.034177
152  1  2      BFS-SA: Building  2.941689
153  1  2                DaC-SP  0.000000
154  1  2                Parlay  0.000000
other-min=2.975269
2.941689
here1 = 2.9
here2 = 2.9
0.03357999999999972
here1 = 0.03
here2 = .03
df_k
     N  K          Algorithm      Time
155  1  3  BFS-B-Hash: Query  0.793010
156  1  3    BFS-Hash: Query  0.468625
157  1  3      BFS-SA: Query  3.200691
158  1  3             DaC-SP  0.000000
159  1  3             Parlay  0.000000
df_k2
     N  K             Algorithm      Time
155  1  3  BFS-B-Hash: Building  0.008391
156  1  3    BFS-Hash: Building  0.037580
157  1  3      BFS

<ipython-input-1-f963fe113f08>:134: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-dark', 'seaborn-paper'])
<ipython-input-1-f963fe113f08>:134: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-dark', 'seaborn-paper'])


size=1000000
[1, 10, 100, 1000, 10000, 100000]
df_k
         N  K          Algorithm       Time
0  1000000  1  BFS-B-Hash: Query   0.000545
1  1000000  1    BFS-Hash: Query   0.001100
2  1000000  1      BFS-SA: Query   0.102103
3  1000000  1             DaC-SP   0.033438
4  1000000  1             Parlay  14.376501
df_k2
         N  K             Algorithm       Time
0  1000000  1  BFS-B-Hash: Building   0.000536
1  1000000  1    BFS-Hash: Building   0.001094
2  1000000  1      BFS-SA: Building   0.101633
3  1000000  1                DaC-SP   0.033438
4  1000000  1                Parlay  14.376501
other-min=0.033438
0.101633
here1 = 0.1
here2 = .1
0.0004699999999999982
here1 = 0.00
here2 = .00
0.033438
here1 = 0.03
here2 = .03
14.376501
here1 = 14
here2 = 14
df_k
         N   K          Algorithm       Time
5  1000000  10  BFS-B-Hash: Query   0.000691
6  1000000  10    BFS-Hash: Query   0.001202
7  1000000  10      BFS-SA: Query   0.101460
8  1000000  10             DaC-SP   0.126446
9 

...

<Figure size 792x108 with 0 Axes>

<Figure size 460.8x316.8 with 0 Axes>

<Figure size 792x108 with 0 Axes>

<Figure size 460.8x316.8 with 0 Axes>

<Figure size 792x108 with 0 Axes>

In [15]:
labels_out

['BFS-B-Hash: Query',
 'BFS-Hash: Query',
 'BFS-SA: Query',
 'BFS-B-Hash: Building',
 'BFS-Hash: Building',
 'BFS-SA: Building',
 'DaC-SP',
 'Parlay']

In [13]:
labels[4:]

['Parlay',
 'BFS-B-Hash: Building',
 'BFS-Hash: Building',
 'BFS-SA: Building',
 'DaC-SP',
 'Parlay']

NameError: name 'plt' is not defined